In [5]:
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import ttest_ind
import math

In [6]:
team_stats1415 = pd.read_csv("./team_stats1415.csv", sep = ",")
team_stats1516 = pd.read_csv("./team_stats1516.csv", sep = ",")
team_stats1617 = pd.read_csv("./team_stats1617.csv", sep = ",")
team_stats1718 = pd.read_csv("./team_stats1718.csv", sep = ",")
scraped_team_stats1718 = pd.read_csv("./scraped_EOS_2017-18.csv", sep = ",")

In [7]:
scraped_team_stats1718.head()

,Unnamed: 0,Team,W,L,Pct,GB,Home,Road,Conf,Div,Streak,L10
0,1,Toronto,59,23,0.720,0,34-7,25-16,40-12,12-4,L1,6-4
1,2,Boston,55,27,0.671,4,27-14,28-13,33-19,12-4,W1,6-4
2,3,Philadelphia,52,30,0.634,7,30-11,22-19,34-18,9-7,W16,10-0
3,4,Cleveland,50,32,0.610,9,29-12,21-20,35-17,11-5,L1,7-3
4,5,Indiana,48,34,0.585,11,27-14,21-20,32-20,10-6,L1,7-3
